In [2]:
from z3 import *

# Parameters
k = 26 # Maximum number of moves
positions = 9  # 3x3 grid
tiles = 8  # Number of tiles


adjacent = {  # Adjacent positions for the 3x3 grid (0-indexed)
    0: [1, 3], 1: [0, 2, 4], 2: [1, 5],
    3: [0, 4, 6], 4: [1, 3, 5, 7], 5: [2, 4, 8],
    6: [3, 7], 7: [4, 6, 8], 8: [5, 7]
}

adjacent_same = {
    0: [0, 1, 3], 1: [0, 1, 2, 4], 2: [1, 2,  5],
    3: [0, 3, 4, 6], 4: [1, 3, 4, 5, 7], 5: [2, 4, 5, 8],
    6: [3, 6, 7], 7: [4, 6, 7, 8], 8: [5, 7, 8]
}


solver = Solver()

# variables
# x[i][j][t]: Boolean variable indicating if tile i is at position j at time t
x = [[[Bool(f"x_{i}_{j}_{t}") for t in range(k+1)] for j in range(positions)] for i in range(tiles)]
# y[j][t]: Boolean variable indicating if the blank space is at position j at time t
y = [[Bool(f"y_{j}_{t}") for t in range(k+1)] for j in range(positions)]

#  Initialization Constraint: Set starting positions
initial_state = [7, 2, 4, 5, None, 6, 8, 3, 1]
for j in range(positions):
    if initial_state[j] is None: # Blank tile
        solver.add(y[j][0])  # Blank is at position j at time 0
    else:
        solver.add(x[initial_state[j]-1][j][0])  # Tile i starts at position j

# Goal State Constraint: Solved configuration at time k
goal_state = [None, 1, 2, 3, 4, 5, 6, 7, 8]
for j in range(positions):
    if goal_state[j] is None:  # Blank tile
        solver.add(y[j][k]) # Blank is at position j at time k
    else:
        solver.add(x[goal_state[j]-1][j][k]) 

#  Uniqueness Constraints
for t in range(k+1):
    # Each position has at most one tile or blank
    for j in range(positions):
        solver.add(Or([y[j][t]] + [x[i][j][t] for i in range(tiles)]))
        for i1 in range(tiles):
            for i2 in range(i1+1, tiles):
                solver.add(Or(Not(x[i1][j][t]), Not(x[i2][j][t])))
        for i in range(tiles):
            solver.add(Or(Not(x[i][j][t]), Not(y[j][t])))
    # Each tile must be in exactly one position
    for i in range(tiles):
        solver.add(Or([x[i][j][t] for j in range(positions)]))
        for j1 in range(positions):
            for j2 in range(j1+1, positions):
                solver.add(Or(Not(x[i][j1][t]), Not(x[i][j2][t])))

# Movement Constraint: Blank tile must move to a neighboring position
for t in range(1, k+1):
    for j in range(positions):

        # Enforce movement: blank cannot stay in the same position
        solver.add(Implies(y[j][t-1], Or([y[n][t] for n in adjacent[j]])))
        solver.add(Implies(y[j][t-1], Not(y[j][t])))
        for i in range(tiles):
            solver.add(Implies(x[i][j][t-1], Implies(Not(x[i][j][t]), y[j][t])))

# Movement Constraint 2: Ensure that tiles move according to the rules of the 8-puzzle
for t in range(1, k+1):
    for j in range(positions):
        for i in range(tiles):
            # Constraint 2.1: If tile i was at position j at time t-1, it must move to one of the adjacent positions at time t
            solver.add(Implies(x[i][j][t-1], Or([x[i][n][t] for n in adjacent_same[j]])))
            # Constraint 2.2: If tile i was at position j at time t-1 but is no longer there at time t,
            # ensure that the other tiles at position j remain in the same position across t-1 and t
            solver.add(Implies(x[i][j][t - 1], Implies(Not(x[i][j][t]),
                                                       And(
                                                           [Implies(x[otherTiles][j][t - 1], x[otherTiles][j][t]) for
                                                            otherTiles in [k for k in range(tiles) if k != i]]))))


if solver.check() == sat:
    print("Solution found!")
    model = solver.model()
    for t in range(k+1):
        print(f"Time step {t}:")
        grid = ['_' for _ in range(positions)]
        for j in range(positions):
            if is_true(model[y[j][t]]):
                grid[j] = 'B'  # Blank tile
            for i in range(tiles):
                if is_true(model[x[i][j][t]]):
                    grid[j] = str(i+1)
        for row in range(3):
            print(" ".join(grid[row*3:(row+1)*3]))
else:
    print("No solution found within given moves.")


Solution found!
Time step 0:
7 2 4
5 B 6
8 3 1
Time step 1:
7 2 4
B 5 6
8 3 1
Time step 2:
B 2 4
7 5 6
8 3 1
Time step 3:
2 B 4
7 5 6
8 3 1
Time step 4:
2 5 4
7 B 6
8 3 1
Time step 5:
2 5 4
7 3 6
8 B 1
Time step 6:
2 5 4
7 3 6
B 8 1
Time step 7:
2 5 4
B 3 6
7 8 1
Time step 8:
2 5 4
3 B 6
7 8 1
Time step 9:
2 5 4
3 6 B
7 8 1
Time step 10:
2 5 B
3 6 4
7 8 1
Time step 11:
2 B 5
3 6 4
7 8 1
Time step 12:
B 2 5
3 6 4
7 8 1
Time step 13:
3 2 5
B 6 4
7 8 1
Time step 14:
3 2 5
6 B 4
7 8 1
Time step 15:
3 2 5
6 4 B
7 8 1
Time step 16:
3 2 5
6 4 1
7 8 B
Time step 17:
3 2 5
6 4 1
7 B 8
Time step 18:
3 2 5
6 4 1
B 7 8
Time step 19:
3 2 5
B 4 1
6 7 8
Time step 20:
3 2 5
4 B 1
6 7 8
Time step 21:
3 2 5
4 1 B
6 7 8
Time step 22:
3 2 B
4 1 5
6 7 8
Time step 23:
3 B 2
4 1 5
6 7 8
Time step 24:
3 1 2
4 B 5
6 7 8
Time step 25:
3 1 2
B 4 5
6 7 8
Time step 26:
B 1 2
3 4 5
6 7 8
